<a href="https://colab.research.google.com/github/NamasteIndia/WZML-X-Original/blob/master/wzmlx_heroku_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <center>𝙒𝙕𝙈𝙇-𝙓 𝙃𝙀𝙍𝙊𝙆𝙐</center><br>
#@markdown <br><center><img src='https://te.legra.ph/file/8086f391e542ed5c6a4c2.jpg'  height="200" width="400" alt="WZML-X-logo"/></center>

#@markdown ---

#@markdown 🔗 **Repo :** https://gitlab.com/mysterysd.sd/WZML-X <br>
#@markdown 🤖 **Version :** _v1.3.3-hx_ <br>
#@markdown ☢️ **Colab Version :** _v1.2_ <br>
#@markdown 📄 **Channel :** https://telegram.me/WZML_X<br>
#@markdown 🫂 **Community :** https://telegram.me/WZML_Support<br>

#@markdown ---
#@markdown <br><b>Deploy WZML-X in Heroku using Google Colab</b><br>

In [ ]:
#@title <center><h3>***Heroku Login***</h3></center><br>

#@markdown ---

Heroku_Email = "" #@param {type:"string"}
Heroku_API = "" #@param {type:"string"}

#@markdown ---

!curl -s https://cli-assets.heroku.com/install.sh | sh

from IPython.display import HTML, clear_output
clear_output()
print("Heroku CLI Installed !")

if not all([Heroku_Email, Heroku_API]):
    raise ValueError("Please fill in all Mandatory Variables.")

from os import path as ospath, chmod

netrc_path = ospath.expanduser("~/.netrc")

netrc_creds = f'''machine api.heroku.com
  login {Heroku_Email}
  password {Heroku_API}
machine git.heroku.com
  login {Heroku_Email}
  password {Heroku_API}'''
with open(netrc_path, "w") as netrc_file:
    netrc_file.write(netrc_creds)

chmod(netrc_path, 0o600)

!git config --global user.email {Heroku_Email}
!git config --global user.name "WZML-X"

print("Heroku Email & Password Loaded!")

In [ ]:
#@title <center><h3>***Create Heroku App***</h3></center><br>

#@markdown ---
App_Name = "" #@param {type:"string"}
Server_Region = "eu" #@param ["eu", "us"] {allow-input: true}
#@markdown ---

!heroku create --region {Server_Region} --stack container {App_Name}

In [ ]:
#@title <center><h3>***WZML-X Repo Config Setup***</h3></center><br>

#@markdown #### ***Fill all these Variables for the `config.env`*** **(All are Mandatory)**

BOT_TOKEN = ""  # @param {type:"string"}
TELEGRAM_API = None  # @param {type:"integer"}
TELEGRAM_HASH = ""  # @param {type:"string"}
OWNER_ID = None  # @param {type:"integer"}
UPSTREAM_REPO = "https://gitlab.com/mysterysd.sd/WZML-X"  # @param {type:"string"}
UPSTREAM_BRANCH = "hk_wzmlx"  # @param {type:"string"}
DATABASE_URL = ""  # @param {type:"string"}
BASE_URL = ""  # @param {type:"string"}
TORRENT_TIMEOUT = 0

#@markdown ---

CONF_GIST_URL = "" # @param {type:"string"}
#@markdown <h6><i>(If you want to Upload `config.env` via `gist.github.com`, Provide the gist URL, Always make Private gist)</i></h6>

#@markdown ---

Upload_Config = False # @param {type:"boolean"}
#@markdown <h6><i>(If you want to Upload `config.env` file Externally, Tick the Above CheckBox)</i></h6>

#@markdown ---

Modify_Pkgs = "yt-dlp==2023.10.07, pyrofork==2.2.11" #@param ["pyrogram==2.0.77", "pyrogram==2.0.106", "bs4, aiohttp, cfscr"] {allow-input: true, type:"string"}
Remove_Pkgs = "pyrogram" #@param ["pyrogram"] {allow-input: true, type:"string"}
#@markdown <h6><i>(If you want to edit requirements.txt custom, separated via ", " (comma) for multiple)</i></h6>

#@markdown ---

if not Upload_Config and not CONF_GIST_URL and not all([BOT_TOKEN, TELEGRAM_API, TELEGRAM_HASH, OWNER_ID, UPSTREAM_REPO, DATABASE_URL, BASE_URL]):
    raise ValueError("Please fill in all Mandatory Variables.")

!git clone https://gitlab.com/mysterysd.sd/WZML-X
%cd WZML-X

if Upload_Config:
    from google.colab import files
    config_creds = list(files.upload().values())[0]
    with open("config.env", "wb") as config_file:
        config_file.write(config_creds)
    print("config.env File Uploaded and Saved Successfully")
elif CONF_GIST_URL:
    !curl -o "config.env" $CONF_GIST_URL
    print("config.env File Extracted and Saved Successfully")
elif all([BOT_TOKEN, TELEGRAM_API, TELEGRAM_HASH, OWNER_ID, UPSTREAM_REPO, DATABASE_URL, BASE_URL]):
    config_creds = f'BOT_TOKEN = "{BOT_TOKEN}"\n' \
                   f'TELEGRAM_API = "{TELEGRAM_API}"\n' \
                   f'TELEGRAM_HASH = "{TELEGRAM_HASH}"\n' \
                   f'OWNER_ID = "{OWNER_ID}"\n' \
                   f'UPSTREAM_REPO = "{UPSTREAM_REPO}"\n' \
                   f'UPSTREAM_BRANCH = "{UPSTREAM_BRANCH}"\n' \
                   f'DATABASE_URL = "{DATABASE_URL}"\n' \
                   f'BASE_URL = "{BASE_URL}"\n' \
                   f'TORRENT_TIMEOUT = "{TORRENT_TIMEOUT}"\n'

    with open("config.env", "wb") as config_file:
        config_file.write(config_creds.encode())
    print("config.env File made and Saved Successfully")

if Modify_Pkgs:
    from re import split as rsplit, escape
    edit_pkgs = [pkg.strip() for pkg in Modify_Pkgs.split(',')]
    rm_pkgs = [pkg.strip() for pkg in Remove_Pkgs.split(',')]

    with open("requirements.txt", "r") as req_file:
        contents = req_file.readlines()

    for i, line in enumerate(contents):
        for pkg in rm_pkgs:
            if line.startswith(pkg.lower()):
                del contents[i]
                rm_pkgs.remove(pkg)

    if len(rm_pkgs) != 0:
        print(f"Following Packages Not Found : {', '.join(rm_pkgs)}")

    for i, line in enumerate(contents):
        for pkg in edit_pkgs:
            if line.startswith(rsplit('|'.join(map(escape, ['==', '>=', '<=', '~='])), pkg)[0]):
                contents[i] = f"{pkg}\n"
                edit_pkgs.remove(pkg)

    if len(edit_pkgs) != 0:
        contents.extend(edit_pkgs)

    with open("requirements.txt", "w") as req_file:
        req_file.writelines(contents)
    print(f"requirements.txt file is Updated : {Modify_Pkgs}")

In [ ]:
# @title <center><h3>***Deploy Heroku App***</h3></center><br>

#@markdown ---
App_Name = "" #@param {type:"string"}
#@markdown ---

!git add . -f
!git commit -m "HK Setup"
!heroku git:remote -a {App_Name}
!git push heroku main -f

In [ ]:
# @title <center><h3>***Show Heroku App Logs***</h3></center><br>

#@markdown ---
App_Name = "" #@param {type:"string"}
#@markdown ---

!heroku logs -a {App_Name}

In [ ]:
#@title <center><h3>***Heroku Logout***</h3></center><br>

!heroku logout